### GUI 꾸미기

In [6]:
import tkinter as tk
from tkinter import filedialog
import pandas as pd

# 마스터 데이터 업데이트하기
def update_master (df, idx, var):
    df['Item'][idx] = var
    df.to_csv('Master/PicMaster.csv', index = False)

# 경로 변경하기
def change_Path (df, idx, fileyn):
    # 파일 경로, 폴더 경로 구분하기
    if fileyn[idx] == 0 :
        f_selected = filedialog.askdirectory(title = lbPath[idx].cget('text'))
        f_selected = f_selected.replace('/','\\')
    else :
        f_selected = filedialog.askopenfile(title = lbPath[idx].cget('text'))
        f_selected = f_selected.name.replace('/','\\')
    # 텍스트 박스에 정보 업데이트하고 마스터 데이터 업데이트
    txtPath[idx].delete('1.0', tk.END)
    txtPath[idx].insert(tk.END, chars=f_selected)
    update_master (df, idx, f_selected)    

# Master Data 가져오기
try : 
    df = pd.read_csv('Master/PicMaster.csv')

except :
    d = {'Item' : ['폴더 경로를 설정해 주세요'] * 5}
    df = pd.DataFrame(data=d)

# GUI 창 생성하고 기본 정보 설정
win = tk.Tk()
win.geometry("680x330")
win.title('사진대지 자동화')

# 폴더/파일 경로 설정 frame 생성
frame1 = tk.Frame(win, padx = 5, pady=5, width = 100)
frame1.pack(padx = 10, pady=10)

# 개체 초기화
lbPath = []
txtPath = []
btnPath = []

# 각 행의 라벨 정보 List로 정리
lbName = ['작업 대상 폴더', '사진 정보 마스터 (엑셀)','사진 정보 저장 폴더', 
'사진대지 양식 (PPT)', '결과 저장 폴더']
# 각 행에서 다루는 값이 폴더일때는 0, 파일일때는 1로 구분자
fileyn = [0, 1, 0, 1, 0]

# 파일 / 폴더 경로 가져오기 GUI 구성
for idx, x in enumerate(lbName):
    # 라벨, 텍스트 상자, 버튼 쌓기
    lbPath.append(tk.Label(frame1, text=x, width = 20, padx = 5, pady=5))
    txtPath.append(tk.Text(frame1, width = 50, height = 1, padx = 5, pady=5, 
                            background='lightgrey'))
    btnPath.append(tk.Button(frame1,text="Path 변경", width = 10,padx = 5, pady=5,
                        command = lambda i = idx : change_Path(df,i,fileyn)))
    # 텍스트 상자 초깃값 넣기
    txtPath[idx].insert(tk.INSERT, chars=df['Item'][idx])

    # grid 배치하기
    lbPath[idx].grid(row=idx, column=0, padx = 5, pady=5)
    txtPath[idx].grid(row=idx, column=1, padx = 5, pady=5)
    btnPath[idx].grid(row=idx, column=2, padx = 5, pady=5)

# 폴더/파일 경로 설정 frame 생성
frame2 = tk.Frame(win, padx = 5, pady=5, width = 100)
frame2.pack(padx = 10, pady=10)

# 버튼 생성하기
btnInitImg = tk.Button(frame2, text = '사진 정보 생성', width = 30, padx=5, pady=5)
btnMPicBoard = tk.Button(frame2, text = '사진대지 자동 생성', width = 30, padx=5, pady=5)

# 버튼 배치하기
btnInitImg.grid(row=6, column=0, padx =5, pady =5)
btnMPicBoard.grid(row=6, column=1, padx =5, pady =5)

win.mainloop()


### 사진 정보 생성하기 기능 함수로 정의

In [ ]:
import openpyxl
import os
import re

# 사진 정보 생성하기
def InitImg(TPath, FPath, RPath):
    # 읽어올 파일 경로 및 폴더 경로 지정 
    # TPath : 타겟 파일 폴더 경로, FPath : 양식 파일, RPath : 결과 폴더
    
    # Image 파일 리스트 불러오기, 확장자명에 jpg, png, gif가 포함된 파일 전체
    files = [f for f in os.listdir(TPath) if re.match('.*([.]jpg|[.]png|[.]gif)', f)]

    # openpyxl 불러오기
    WB = openpyxl.load_workbook(FPath)

    # 활성화된 시트를 WS로 지정하고 안에 있는 기존 이미지 모두 지우기
    WS = WB.active
    WS._images = []

    # 행 초깃값 설정 (2번째 행부터 값 쓰기)
    i=2

    # 파일 리스트에서 값 반복
    for file in files:
        # 폴더 이름과 파일명 입력
        WS["A" + str(i)].value = TPath
        WS["B" + str(i)].value = file

        # 이미지 불러오기
        F_Name = os.path.join(TPath,file)
        img = openpyxl.drawing.image.Image(F_Name)
        
        # 이미지 가로 세로 비율 구하기
        I_Rate = img.width / img.height

        # 이미지 폭을 F열 폭에 맞추고 F열의 칸에 위치 시키기
        img.width = WS.column_dimensions['F'].width * 8
        img.height = img.width / I_Rate
        img.anchor = 'F' + str(i)

        # 이미지 높이에 행의 높이 맞추기
        WS.row_dimensions[i].height = img.height * 0.75

        # 이미지 확정하여 삽입하기
        WS.add_image(img)

        # 작업 줄 증가
        i += 1

    # 결과 저장하기
    WB.save(os.path.join(RPath, '사진정보.xlsx'))


### 사진 정보 생성하기 기능 함수로 정의 (작업 완료후 엑셀 파일 열기 기능 추가)

In [ ]:
import openpyxl
import os
import re
import win32com.client

# 사진 정보 생성하기
def InitImg(TPath, FPath, RPath):
    # 읽어올 파일 경로 및 폴더 경로 지정 
    # TPath : 타겟 파일 폴더 경로, FPath : 양식 파일, RPath : 결과 폴더
    
    # Image 파일 리스트 불러오기, 확장자명에 jpg, png, gif가 포함된 파일 전체
    files = [f for f in os.listdir(TPath) if re.match('.*([.]jpg|[.]png|[.]gif)', f)]

    # openpyxl 불러오기
    WB = openpyxl.load_workbook(FPath)

    # 활성화된 시트를 WS로 지정하고 안에 있는 기존 이미지 모두 지우기
    WS = WB.active
    WS._images = []

    # 행 초깃값 설정 (2번째 행부터 값 쓰기)
    i=2

    # 파일 리스트에서 값 반복
    for file in files:
        # 폴더 이름과 파일명 입력
        WS["A" + str(i)].value = TPath
        WS["B" + str(i)].value = file

        # 이미지 불러오기
        F_Name = os.path.join(TPath,file)
        img = openpyxl.drawing.image.Image(F_Name)
        
        # 이미지 가로 세로 비율 구하기
        I_Rate = img.width / img.height

        # 이미지 폭을 F열 폭에 맞추고 F열의 칸에 위치 시키기
        img.width = WS.column_dimensions['F'].width * 8
        img.height = img.width / I_Rate
        img.anchor = 'F' + str(i)

        # 이미지 높이에 행의 높이 맞추기
        WS.row_dimensions[i].height = img.height * 0.75

        # 이미지 확정하여 삽입하기
        WS.add_image(img)

        # 작업 줄 증가
        i += 1

    # 결과 저장하기
    RFile = os.path.join(RPath, '사진정보.xlsx')
    WB.save(RFile)

    # 엑셀 애플리케이션 개체 만들기
    excel = win32com.client.Dispatch("Excel.Application")
    # 엑셀 애플리케이션 보이게 하고 파일 열기
    excel.Visible = True
    WB = excel.Workbooks.Open(RFile)


### 사진 대지 만들기(PPT) 기능 함수로 정의

In [ ]:
from pptx import Presentation

def MPicBoard(TPath, PMaster, RPath):
    # 사용할 파일 경로 설정, PMaster : 사진대지 마스터 ppt파일, 
# TPath : 사진대지 정보 엑셀 파일
    TPath = os.path.join(TPath,'사진정보.xlsx')
    RPath = os.path.join(RPath,'사진대지.pptx')

    # 사진대지 정보 읽어오기
    df = pd.read_excel(TPath)

    # 파워포인트 개체 열기
    prs = Presentation(PMaster)

    # 0번 슬라이드 레이아웃 번호 지정하기
    s_layout = prs.slide_layouts[0]


    # 읽어온 사진대지 정보에 대하여 반복문
    for index, row in df.iterrows():
        # 홀수번째 이미지인지 짝수번째 정보인지 판단하여 구분자 r로 지정 0: 홀수번째
        r = index % 2
        # 홀수번째 이미지이면 슬라이드를 한장 추가
        if r == 0 :
            sld = prs.slides.add_slide(s_layout)
        # 슬라이드의 정해진 개체틀에 필요한 정보 채워넣기
        sld.placeholders[10 + r * 4].insert_picture(os.path.join(row['폴더'],row['파일명']))
        sld.placeholders[11 + r * 4].text = str(row['내용'])
        sld.placeholders[12 + r * 4].text = str(row['위치'])
        sld.placeholders[13 + r * 4].text = str(row['일시'])
        
    # 결과 저장
    prs.save(RPath)
    
    # 파워포인트 애플리케이션 개체 만들기
    powerpoint = win32com.client.Dispatch("Powerpoint.Application")

    # 파워포인트에서 파일 읽어오기
    prs = powerpoint.Presentations.Open(RPath)


### 전체 소스 합치기(최종본)

In [1]:
import openpyxl
import os
import re
import win32com.client
from pptx import Presentation
import tkinter as tk
from tkinter import filedialog
import pandas as pd

# 사진 정보 생성하기
def InitImg(TPath, FPath, RPath):
    # 읽어올 파일 경로 및 폴더 경로 지정 
    # TPath : 타겟 파일 폴더 경로, FPath : 양식 파일, RPath : 결과 폴더
    
    # Image 파일 리스트 불러오기, 확장자명에 jpg, png, gif가 포함된 파일 전체
    files = [f for f in os.listdir(TPath) if re.match('.*([.]jpg|[.]png|[.]gif)', f)]

    # openpyxl 불러오기
    WB = openpyxl.load_workbook(FPath)

    # 활성화된 시트를 WS로 지정하고 안에 있는 기존 이미지 모두 지우기
    WS = WB.active
    WS._images = []

    # 행 초깃값 설정 (2번째 행부터 값 쓰기)
    i=2

    # 파일 리스트에서 값 반복
    for file in files:
        # 폴더 이름과 파일명 입력
        WS["A" + str(i)].value = TPath
        WS["B" + str(i)].value = file

        # 이미지 불러오기
        F_Name = os.path.join(TPath,file)
        img = openpyxl.drawing.image.Image(F_Name)
        
        # 이미지 가로 세로 비율 구하기
        I_Rate = img.width / img.height

        # 이미지 폭을 F열 폭에 맞추고 F열의 칸에 위치 시키기
        img.width = WS.column_dimensions['F'].width * 8
        img.height = img.width / I_Rate
        img.anchor = 'F' + str(i)

        # 이미지 높이에 행의 높이 맞추기
        WS.row_dimensions[i].height = img.height * 0.75

        # 이미지 확정하여 삽입하기
        WS.add_image(img)

        # 작업 줄 증가
        i += 1

    # 결과 저장하기
    RFile = os.path.join(RPath, '사진정보.xlsx')
    WB.save(RFile)

    # 엑셀 애플리케이션 개체 만들기
    excel = win32com.client.Dispatch("Excel.Application")
    # 엑셀 애플리케이션 보이게 하고 파일 열기
    excel.Visible = True
    wb = excel.Workbooks.Open(RFile)

def MPicBoard(TPath, PMaster, RPath):
    # 사용할 파일 경로 설정, PMaster : 사진대지 마스터 ppt파일, 
# TPath : 사진대지 정보 엑셀 파일
    TPath = os.path.join(TPath,'사진정보.xlsx')
    RPath = os.path.join(RPath,'사진대지.pptx')

    # 사진대지 정보 읽어오기
    df = pd.read_excel(TPath)

    # 파워포인트 개체 열기
    prs = Presentation(PMaster)

    # 0번 슬라이드 레이아웃 번호 지정하기
    s_layout = prs.slide_layouts[0]

    # 읽어온 사진대지 정보에 대하여 반복문
    for index, row in df.iterrows():
        # 홀수번째 이미지인지 짝수번째 정보인지 판단하여 구분자 r로 지정 0: 홀수번째
        r = index % 2
        # 홀수번째 이미지이면 슬라이드를 한장 추가
        if r == 0 :
            sld = prs.slides.add_slide(s_layout)
        # 슬라이드의 정해진 개체틀에 필요한 정보 채워넣기
        sld.placeholders[10 + r * 4].insert_picture(os.path.join(row['폴더'],row['파일명']))
        sld.placeholders[11 + r * 4].text = str(row['내용'])
        sld.placeholders[12 + r * 4].text = str(row['위치'])
        sld.placeholders[13 + r * 4].text = str(row['일시'])
        
    # 결과 저장
    prs.save(RPath)
    
    # 파워포인트 애플리케이션 개체 만들기
    powerpoint = win32com.client.Dispatch("Powerpoint.Application")

    # 파워포인트에서 파일 읽어오기
    prs = powerpoint.Presentations.Open(RPath)


# 마스터 데이터 업데이트하기
def update_master (df, idx, var):
    df['Item'][idx] = var
    df.to_csv('Master/PicMaster.csv', index = False)

# 경로 변경하기
def change_Path (df, idx, fileyn):
    # 파일 경로, 폴더 경로 구분하기
    if fileyn[idx] == 0 :
        f_selected = filedialog.askdirectory(title = lbPath[idx].cget('text'))
        f_selected = f_selected.replace('/','\\')
    else :
        f_selected = filedialog.askopenfile(title = lbPath[idx].cget('text'))
        f_selected = f_selected.name.replace('/','\\')
    # 텍스트 박스에 정보 업데이트하고 마스터 데이터 업데이트
    txtPath[idx].delete('1.0', tk.END)
    txtPath[idx].insert(tk.END, chars=f_selected)
    update_master (df, idx, f_selected)    

# Master Data 가져오기
try : 
    df = pd.read_csv('Master/PicMaster.csv')

except :
    d = {'Item' : ['폴더 경로를 설정해 주세요'] * 5}
    df = pd.DataFrame(data=d)

# GUI 창 생성하고 기본 정보 설정
win = tk.Tk()
win.geometry("680x330")
win.title('사진대지 자동화')

# 폴더/파일 경로 설정 frame 생성
frame1 = tk.Frame(win, padx = 5, pady=5, width = 100)
frame1.pack(padx = 10, pady=10)

# 개체 초기화
lbPath = []
txtPath = []
btnPath = []

# 각 행의 라벨 정보 List로 정리
lbName = ['작업 대상 폴더', '사진 정보 마스터 (엑셀)','사진 정보 저장 폴더', 
'사진대지 양식 (PPT)', '결과 저장 폴더']
# 각 행에서 다루는 값이 폴더일때는 0, 파일일때는 1로 구분자
fileyn = [0, 1, 0, 1, 0]

# 파일 / 폴더 경로 가져오기 GUI 구성
for idx, x in enumerate(lbName):
    # 라벨, 텍스트 상자, 버튼 쌓기
    lbPath.append(tk.Label(frame1, text=x, width = 20, padx = 5, pady=5))
    txtPath.append(tk.Text(frame1, width = 50, height = 1, padx = 5, pady=5, 
                            background='lightgrey'))
    btnPath.append(tk.Button(frame1,text="Path 변경", width = 10,padx = 5, pady=5,
                        command = lambda i = idx : change_Path(df,i,fileyn)))
    # 텍스트 상자 초깃값 넣기
    txtPath[idx].insert(tk.INSERT, chars=df['Item'][idx])

    # grid 배치하기
    lbPath[idx].grid(row=idx, column=0, padx = 5, pady=5)
    txtPath[idx].grid(row=idx, column=1, padx = 5, pady=5)
    btnPath[idx].grid(row=idx, column=2, padx = 5, pady=5)

# 폴더/파일 경로 설정 frame 생성
frame2 = tk.Frame(win, padx = 5, pady=5, width = 100)
frame2.pack(padx = 10, pady=10)

# 버튼 생성하기
btnInitImg = tk.Button(frame2, text = '사진 정보 생성', width = 30, padx=5, pady=5,
                        command = lambda : InitImg(df['Item'][0],df['Item'][1],df['Item'][2]))
btnMPicBoard = tk.Button(frame2, text = '사진대지 자동 생성', width = 30, padx=5, pady=5,
                        command = lambda : MPicBoard(df['Item'][2],df['Item'][3],df['Item'][4]))

# 버튼 배치하기
btnInitImg.grid(row=6, column=0, padx =5, pady =5)
btnMPicBoard.grid(row=6, column=1, padx =5, pady =5)

win.mainloop()
